In [ ]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
from langchain.chat_models import init_chat_model

In [37]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

In [38]:
#message = {"role": "user","content": "Who Walked on moon for first time? Print only name"}
#response = graph.invoke({"messages": [message]})
#print(response)



{'messages': [HumanMessage(content='Who Walked on moon for first time? Print only name', additional_kwargs={}, response_metadata={}, id='0b304cc2-a5c4-4693-a32a-33244c40612a'), AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f1cf5a26-b7aa-4a3a-af83-17d8a0df425c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 3, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})]}
